# Préparation des données de maintenance

Chargement des donnnées

In [1]:
import pandas as pd
import numpy  as np

#Chargement des données
maintenance = pd.read_csv("../data/raw/PdM_maint.csv")
# transformer les dates dans le bon type
maintenance["datetime"] = pd.to_datetime(maintenance["datetime"])

#Création des indexes
maintenance = maintenance.set_index(['machineID', 'datetime'])
maintenance = pd.get_dummies(maintenance["comp"])
maintenance = maintenance.groupby(["machineID","datetime"]).sum()

In [2]:
maintenance.tail(8)

comp1  comp2  comp3  comp4
machineID datetime                                       
100       2015-08-26 06:00:00      0      0      0      1
          2015-09-10 06:00:00      1      0      0      0
          2015-09-25 06:00:00      1      0      0      0
          2015-10-10 06:00:00      1      0      1      0
          2015-10-25 06:00:00      0      0      0      1
          2015-11-09 06:00:00      0      0      0      1
          2015-12-09 06:00:00      0      1      0      0
          2015-12-24 06:00:00      0      1      0      0

Toutes les maintenances sont enregistrées à 06:00H, depuis le 1er Juin 2014. Pour chaque  composant, on calcule la durée en jours depuis  le dernier changement. On commence par créer les indexes machines/temps nécessaires

In [3]:
machines = pd.read_csv("../data/raw/PdM_machines.csv")

time_series = pd.date_range(start = "2014-06-01 06:00:00", end = "2016-01-01 06:00:00", freq="1D")
indexes = pd.MultiIndex.from_product([machines["machineID"].values,time_series.values], names=['machineID', 'datetime'])

On fait une jointure de ces indexes avec les données de maintenance

In [4]:
maintenance = pd.DataFrame(index = indexes).join(maintenance).fillna(0)

Une boucle pour compter les jours, pour chaque machine et chaque composants

In [5]:
days_since_maintenance1 = []
days_since_maintenance2 = []
days_since_maintenance3 = []
days_since_maintenance4 = []
for machine in machines["machineID"].values:
    days_count = 0
    for reset in maintenance.loc[machine]["comp1"]:
        days_count = (days_count+1)*(1-reset)
        days_since_maintenance1.append(days_count)
    days_count = 0
    for reset in maintenance.loc[machine]["comp2"]:
        days_count = (days_count+1)*(1-reset)
        days_since_maintenance2.append(days_count)
    days_count = 0
    for reset in maintenance.loc[machine]["comp3"]:
        days_count = (days_count+1)*(1-reset)
        days_since_maintenance3.append(days_count)
    days_count = 0
    for reset in maintenance.loc[machine]["comp4"]:
        days_count = (days_count+1)*(1-reset)
        days_since_maintenance4.append(days_count)
        
maintenance["comp1_delay"] = days_since_maintenance1
maintenance["comp2_delay"] = days_since_maintenance2
maintenance["comp3_delay"] = days_since_maintenance3
maintenance["comp4_delay"] = days_since_maintenance4

On peut effacer les colonnes comp1 à comp4, et ensuite on va dupliquer les valeurs pour avoir un point toutes les 3h. On peut maintenant restreindre aux dates à partir du 1er Janvier 2015

In [6]:
maintenance.drop(columns=["comp1","comp2","comp3","comp4"], inplace = True)

time_series = pd.date_range(start = "2015-01-01 06:00:00", end = "2016-01-01 06:00:00", freq="3H")
indexes = pd.MultiIndex.from_product([machines["machineID"].values,time_series.values], names=['machineID', 'datetime'])
maintenance = pd.DataFrame(index = indexes).join(maintenance).fillna(method = 'ffill')
maintenance.head(20)

comp1_delay  comp2_delay  comp3_delay  \
machineID datetime                                                     
1         2015-01-01 06:00:00         19.0        214.0        154.0   
          2015-01-01 09:00:00         19.0        214.0        154.0   
          2015-01-01 12:00:00         19.0        214.0        154.0   
          2015-01-01 15:00:00         19.0        214.0        154.0   
          2015-01-01 18:00:00         19.0        214.0        154.0   
          2015-01-01 21:00:00         19.0        214.0        154.0   
          2015-01-02 00:00:00         19.0        214.0        154.0   
          2015-01-02 03:00:00         19.0        214.0        154.0   
          2015-01-02 06:00:00         20.0        215.0        155.0   
          2015-01-02 09:00:00         20.0        215.0        155.0   
          2015-01-02 12:00:00         20.0        215.0        155.0   
          2015-01-02 15:00:00         20.0        215.0        155.0   
          2015-01-02 18:00:00         20.0        215.0        155.0   
          2015-01-02 21:00:00         20.0        215.0        155.0   
          2015-01-03 00:00:00         20.0        215.0        155.0   
          2015-01-03 03:00:00         20.0        215.0        155.0   
          2015-01-03 06:00:00         21.0        216.0        156.0   
          2015-01-03 09:00:00         21.0        216.0        156.0   
          2015-01-03 12:00:00         21.0        216.0        156.0   
          2015-01-03 15:00:00         21.0        216.0        156.0   

                               comp4_delay  
machineID datetime                          
1         2015-01-01 06:00:00        169.0  
          2015-01-01 09:00:00        169.0  
          2015-01-01 12:00:00        169.0  
          2015-01-01 15:00:00        169.0  
          2015-01-01 18:00:00        169.0  
          2015-01-01 21:00:00        169.0  
          2015-01-02 00:00:00        169.0  
          2015-01-02 03:00:00        169.0  
          2015-01-02 06:00:00        170.0  
          2015-01-02 09:00:00        170.0  
          2015-01-02 12:00:00        170.0  
          2015-01-02 15:00:00        170.0  
          2015-01-02 18:00:00        170.0  
          2015-01-02 21:00:00        170.0  
          2015-01-03 00:00:00        170.0  
          2015-01-03 03:00:00        170.0  
          2015-01-03 06:00:00        171.0  
          2015-01-03 09:00:00        171.0  
          2015-01-03 12:00:00        171.0  
          2015-01-03 15:00:00        171.0

In [7]:
maintenance.to_csv("../data/prepared_data/maintenance.csv")